데이터셋은 우선적으로는 CelebDFV2를 사용하고, 이후에 GTA-V 데이터를 합쳐보도록 하자.

이걸 어디부터 구현해야 할까...

일단 SigLIP을 불러오는 것부터 시작하자. Quantize를 해줘서 최대한 부담을 줄여주자.

In [1]:
!uv pip install --system flash-attn --no-build-isolation
!pip install -q flash-attn

Using Python 3.12.12 environment at: /usr
Resolved 28 packages in 1.06s
Prepared 1 package in 9.86s
Installed 1 package in 2ms
 + flash-attn==2.8.3


In [2]:
!uv pip install --system lightning
!pip install -q lightning

Using Python 3.12.12 environment at: /usr
Resolved 43 packages in 271ms
Prepared 4 packages in 82ms
Installed 4 packages in 10ms
 + lightning==2.6.1
 + lightning-utilities==0.15.2
 + pytorch-lightning==2.6.1
 + torchmetrics==1.8.2


In [3]:
!ffmpeg -decoders | grep -i nvidia

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [4]:
!uv pip install --system bitsandbytes
!pip install -q bitsandbytes

Using Python 3.12.12 environment at: /usr
Resolved 29 packages in 47ms
Prepared 1 package in 685ms
Installed 1 package in 5ms
 + bitsandbytes==0.49.1


In [5]:
!uv pip install --force-reinstall torchcodec==0.9.0 --index-url=https://download.pytorch.org/whl/cu126

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 940ms
Prepared 1 package in 43ms
Installed 1 package in 3ms
 + torchcodec==0.9.0+cu126


In [1]:
import lightning.pytorch as L
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModel, BitsAndBytesConfig

embed_size = 768

frame_token = 576

솔직히, 왠만하면 원본 모델을 따라가고 싶지만... 일단 인코더를 더 경량으로 바꾼다.

나중에 성능 안 나오면 탓할 것 중 인코더가 늘었다.

In [2]:
import torch.nn as nn

In [3]:
import torch.nn.functional as F
from flash_attn import flash_attn_func

class ViTEncoder(nn.Module):
    def __init__(self, embed_size=768, num_heads=12, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = embed_size // num_heads
        assert self.head_dim * num_heads == embed_size, "embed_size must be divisible by num_heads"

        self.q_proj = nn.Linear(embed_size, embed_size)
        self.k_proj = nn.Linear(embed_size, embed_size)
        self.v_proj = nn.Linear(embed_size, embed_size)
        self.out_proj = nn.Linear(embed_size, embed_size)
        self.dropout_p = dropout

        self.ln1 = nn.LayerNorm(embed_size)
        self.mlp = nn.Sequential(
            nn.Linear(embed_size, 4 * embed_size),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(4 * embed_size, embed_size),
            nn.Dropout(dropout),
        )
        self.ln2 = nn.LayerNorm(embed_size)

    def forward(self, x, return_attn_output=False):
        # x: (batch * frames, token/frame, embed_size)
        batch_size, frame_token, embed_size = x.shape

        # Project queries, keys, values
        q = self.q_proj(x) # (batch * frames, token/frame, embed_size)
        k = self.k_proj(x) # (batch * frames, token/frame, embed_size)
        v = self.v_proj(x) # (batch * frames, token/frame, embed_size)

        # Split into multiple heads
        q = q.view(batch_size, frame_token, self.num_heads, self.head_dim) # (batch * frames, token/frame, num_heads, head_dim)
        k = k.view(batch_size, frame_token, self.num_heads, self.head_dim) # (batch * frames, token/frame, num_heads, head_dim)
        v = v.view(batch_size, frame_token, self.num_heads, self.head_dim) # (batch * frames, token/frame, num_heads, head_dim)

        # Apply scaled dot product attention
        # dropout_p는 훈련 중일 때만 적용
        attn_output_raw = flash_attn_func(
            q, k, v,
            dropout_p=self.dropout_p if self.training else 0.0,
            softmax_scale=None, # None이면 1/sqrt(head_dim) 자동 적용
            causal=False
        )
        # attn_output: (batch * frames, token/frame, num_heads, head_dim)

        # Concatenate heads and apply final linear projection
        attn_output = attn_output_raw.contiguous().view(batch_size, frame_token, embed_size)
        attn_output = self.out_proj(attn_output) # (batch_size, seq_len, embed_size)


        x = self.ln1(x + attn_output) # Residual connection + LayerNorm
        x = self.ln2(x + self.mlp(x))

        if return_attn_output:
            return x, attn_output_raw
        return x

In [4]:
import math


class TemporalPositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 32, dropout: float = 0.1):
        """
        Args:
            d_model: Feature dimension (ds = 768)
            max_len: Maximum number of frames (nf = 32)
            dropout: Dropout probability
        """
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 1. Frame Index (j) 생성: 0 ~ max_len-1
        # 논문의 'j'에 해당합니다.
        position = torch.arange(max_len).unsqueeze(1).float()  # [max_len, 1]

        # 2. Div Term 계산 (논문 Eq 1의 분모 부분)
        # 10000^(2i/ds) 부분을 로그 스케일로 계산
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # 3. PE Matrix 초기화 [max_len, d_model]
        pe = torch.zeros(max_len, d_model)

        # 4. 논문 수식 (Eq 1) 적용
        # PE(j, 2i) = sin(...) -> 짝수 인덱스
        pe[:, 0::2] = torch.sin(position * div_term)
        # PE(j, 2i+1) = cos(...) -> 홀수 인덱스
        pe[:, 1::2] = torch.cos(position * div_term)

        # 5. 차원 확장 (Broadcasting 준비)
        # PE는 [Frame, Dim] 정보를 담고 있습니다.
        # 입력 x: [Batch, Frames, Tokens, Dim]
        # PE  : [1,     Frames, 1,      Dim] 형태로 만들어야
        # Batch와 Tokens 차원으로 자동 확장(Broadcast)되어 더해집니다.
        pe = pe.unsqueeze(0).unsqueeze(2)

        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: [Batch, Frames, Tokens, Dim]
               (예: [B, 32, 576, 768])
        Returns:
            x: Temporal PE가 더해진 텐서
        """
        # 입력된 비디오의 실제 프레임 수만큼만 PE를 잘라서 사용
        current_frames = x.size(1)

        # x에 PE를 더함.
        # pe[:, :current_frames, :, :]의 shape은 [1, F, 1, D]
        # x의 shape [B, F, T, D]에 맞춰서,
        # 같은 프레임(F)에 있는 모든 토큰(T)들에게 동일한 PE 벡터가 더해짐.
        x = x + self.pe[:, :current_frames, :, :]

        return self.dropout(x)

In [5]:
import torch
from torchvision.transforms import v2

class GPUSigLIPProcessor:
    def __init__(self, processor):
        config = processor.image_processor

        # 1. 리사이즈 설정: Bilinear + Antialias=True가 핵심
        # Fast 프로세서가 텐서를 처리할 때 사용하는 로직과 일치시킵니다.
        self.resize = v2.Resize(
            size=(config.size['height'], config.size['width']),
            interpolation=v2.InterpolationMode.BILINEAR, # resample=2
            antialias=True # 오차를 줄이는 가장 중요한 설정
        )

        # 2. 정규화 설정
        # (x - 0.5) / 0.5 연산
        self.mean = torch.tensor(config.image_mean).view(1, 3, 1, 1)
        self.std = torch.tensor(config.image_std).view(1, 3, 1, 1)
        self.rescale_factor = config.rescale_factor

    def __call__(self, video_tensor):
        """
        video_tensor: (B, 3, T, H, W), uint8, GPU
        """
        b, c, t, h, w = video_tensor.shape
        device = video_tensor.device

        # 차원 변경 (B*T, C, H, W)
        x = video_tensor.permute(0, 2, 1, 3, 4)
        x = x.flatten(0, 1)

        # [Step 1] Resize (uint8 상태에서 수행하거나 float32에서 수행)
        # torchvision v2는 uint8 입력을 받아 내부적으로 고정밀 연산을 수행합니다.
        x = self.resize(x)

        # [Step 2] Float32 변환 및 Rescale (0~255 -> 0~1)
        x = x.to(torch.float32) * self.rescale_factor

        # [Step 3] Normalize (x - 0.5) / 0.5
        # mean, std를 캐싱하여 속도 최적화
        self.mean = self.mean.to(device)
        self.std = self.std.to(device)
        x = (x - self.mean) / self.std

        # [Step 4] 최종 모델 입력형태인 float16으로 반환
        return x.to(torch.float16)

In [6]:
class UNITE(nn.Module):
    def __init__(
            self, num_channel=3, num_cls=2, num_heads=12, max_len=32, dropout=0.1,
            encoder_model="google/siglip2-base-patch16-384", use_bfloat=True, cpu_preprocess=True
        ):
        super().__init__()

        dtype = torch.bfloat16 if use_bfloat else torch.float16
        self.vis_encoder = AutoModel.from_pretrained(
            encoder_model,
            device_map="auto",
            dtype=dtype,
            attn_implementation="flash_attention_2",
        )
        self.embed_size = self.vis_encoder.config.vision_config.hidden_size
        processor = AutoProcessor.from_pretrained(encoder_model, use_fast=True)
        self.processor = GPUSigLIPProcessor(processor)
        self.num_heads = num_heads
        self.cpu_preprocess = cpu_preprocess

        for para in self.vis_encoder.parameters():
            para.requires_grad = False
        self.vis_encoder.eval()

        self.class_token = nn.Parameter(torch.randn((self.embed_size,)), requires_grad=True)

        self.pos_embedding = TemporalPositionalEncoding(self.embed_size, max_len)
        self.first_encoder = ViTEncoder(self.embed_size, num_heads, dropout)
        self.encoders = nn.ModuleList([ViTEncoder(self.embed_size, num_heads, dropout) for _ in range(3)])
        self.mlp_head = nn.Linear(self.embed_size, num_cls)


    def forward(self, x, return_ad_param=False, return_embed=False):
        self.vis_encoder.eval()

        # Input: [batch, c, frame, h, w]
        b, _, f, *_ = x.shape

        with torch.no_grad():
            if self.cpu_preprocess:
                x = x.permute(0, 2, 1, 3, 4).flatten(0, 1) # (B, C, F, H, W) -> (B*F, C, H, W)
                x = x.to(self.vis_encoder.dtype)
            else:
                # -> Preprocessing [batch * frame, c, h, w]
                x = self.processor(x)
            # -> Visual encoding [batch * frame, token/frame(576), dim/token (embed_size)]
            x = self.vis_encoder.vision_model(pixel_values=x).last_hidden_state
        # -> [batch, frame, token/frame, dim/token]
        x = x.reshape(b, f, -1, self.embed_size)
        x = self.pos_embedding(x)
        train_in = x # xi: [batch, frame, token/frame, dim/token]

        _, _, t, d = x.shape
        # Reshape for transformer
        # -> [batch * frame, token/frame, dim/token]
        x = x.reshape(b*f, t, d)
        # Add class token [batch*frame, 1, dim/token]
        cls_token = self.class_token.view(1, 1, -1).expand(b*f, -1, -1)
        x = torch.cat([cls_token, x], dim=1)
        P = None
        if return_ad_param:
            # attn_output: [batch*frame, token/frame + class token, num_head, (dim/token) / num_head]
            # x: [batch*frame, token/frame + class_token, dim/token]
            x, attn_output = self.first_encoder(x, return_attn_output=True)
            # -> [batch*frame, num_head, token/frame, (dim/token) / num_head]
            attn_output = attn_output.permute(0, 2, 1, 3)[:, :, 1:, :]
            # xi -> [batch*frame, token/frame, num_head, (dim/token) / num_head] -> [batch*frame, num_head, token/frame, (dim/token) / num_head]
            train_in = train_in.reshape(-1, t, self.num_heads, d // self.num_heads).permute(0, 2, 1, 3)
            P = attn_output * train_in
            # [batch*frame, num_head, token/frame, (dim/token) / num_head] -> [batch*frame, num_head]
            P = P.sum(dim=(2, 3))
            # -> [batch, frame, num_head] -> [batch, num_head, frame]
            P = P.reshape(b, f, -1).permute(0, 2, 1)
        else:
            x = self.first_encoder(x)

        for encoder in self.encoders:
            x = encoder(x)

        # Get only cls_token
        # [batch * frame, 1, dim/token]
        x = x[:, 0, :]
        # [batch, frame, dim/token]
        x = x.reshape(b, f, -1)
        # [batch, dim/token]: Temporal Avg. Pooling
        x = x.mean(dim=1)
        embed = x
        x = self.mlp_head(x)
        res = [x]
        if return_ad_param:
            res.append(P)
        if return_embed:
            res.append(embed)
        return tuple(res) if len(res) > 1 else res[0]

In [7]:
class ADLoss(nn.Module):
    def __init__(self, num_cls=2, num_heads=12, max_len=32, delta_within=(0.01, -2.0), delta_between=0.5, eta=0.05):
        super().__init__()
        # C shape: [num_classes, num_heads, max_len]
        # 논문 식(3)에 따라 센터를 각 클래스별로 유지해야 함
        C = torch.zeros(num_cls, num_heads, max_len)
        self.register_buffer('C', C)

        self.num_cls = num_cls
        self.num_heads = num_heads
        self.delta_within = torch.tensor(delta_within) # [0.01, -2.0] (True, Fake)
        self.delta_between = delta_between # 0.5
        self.eta = eta

    def forward(self, P, labels, log_detail=False):
        """
        P: [batch, num_heads, max_len]
        labels: [batch] (Class indices)
        """
        device = P.device
        # P: [B, H, F] -> [B, H*F]로 정규화하는 게 아니라,
        # 각 헤드(H)별로 정규화해야 논문의 '헤드별 센터' 개념에 맞습니다.
        P_norm = F.normalize(P, p=2, dim=2) # 각 헤드의 특징 벡터(F)를 정규화

        # 1. 센터 업데이트 (현재 로직 유지하되 헤드별로 정규화 상태 유지)
        if self.training:
            for c in range(self.num_cls):
                mask = (labels == c)
                if mask.any():
                    batch_class_mean = P_norm[mask].mean(dim=0) # [H, F]
                    with torch.no_grad():
                        self.C[c] = (1 - self.eta) * self.C[c] + self.eta * batch_class_mean.detach()

        # 센터 정규화 (거리 계산 전 필수)
        C_norm = F.normalize(self.C, p=2, dim=2) # [num_cls, H, F]

        # --- 2. Within-class Loss (식 4) ---
        # 각 샘플과 자기 클래스 센터 사이의 거리
        # P: [B, H, F], self.C[labels]: [B, H, F]
        diff_within = P_norm - C_norm[labels]
        # L2 Norm 계산 (헤드와 프레임 차원에 대해)
        dist_within = torch.norm(diff_within, p=2, dim=(1, 2))

        # 각 샘플별 delta 적용
        # loss_within = torch.relu(dist_within - self.delta_within[labels]).mean()

        # 샘플 수와 상관없이 일정한 스케일 유지
        loss_sum = 0.0
        for c in range(self.num_cls):
            mask = (labels == c)
            if mask.any():
                # 해당 클래스 샘플들만의 평균을 구함
                class_loss = torch.relu(dist_within[mask] - self.delta_within[c]).mean()
                loss_sum += class_loss
        loss_within = loss_sum / self.num_cls # 클래스당 기여도를 1/N로 고정


        # 3. Between-class Loss -> 사실상 "Between-Head Diversity Loss"
        # 논문 식 (5)의 (k, l) in (nh, nh)를 구현
        loss_between = 0.0
        for c in range(self.num_cls):
            # 한 클래스의 센터 내에 있는 12개 헤드 간의 거리를 계산
            class_centers = C_norm[c] # [H, F]

            # 헤드 간 Pairwise 거리: [H, H]
            dist_matrix = torch.cdist(class_centers, class_centers, p=2)

            # k != l 인 상삼각 행렬 마스크
            mask = torch.triu(torch.ones(self.num_heads, self.num_heads, device=device), diagonal=1).bool()
            different_heads_dist = dist_matrix[mask]

            # 헤드들이 최소 delta_between만큼은 떨어져 있도록 함
            loss_between += torch.relu(self.delta_between - different_heads_dist).mean()

        loss_between /= self.num_cls # 클래스별 평균

        if log_detail:
            return loss_within + loss_between, loss_within, loss_between

        return loss_within + loss_between

In [8]:
from torchmetrics.classification import Accuracy, AveragePrecision, Precision, Recall
from torchmetrics import MetricCollection

In [9]:
from typing import TypedDict, Literal
from torchmetrics import ConfusionMatrix
import seaborn as sns
import matplotlib.pyplot as plt

class LitUNITEClassifier(L.LightningModule):
    def __init__(
            self, num_cls=2, num_heads=12, max_len=32, dropout=0.1,
            encoder_model="google/siglip2-base-patch16-384", use_bfloat=True, cpu_preprocess=True,
            delta_within=(0.01, -2.0), delta_between=0.5, eta=0.05,
            lambda_1=0.5, lambda_2=0.5, lr=1e-4, decay_steps=1000,
        ):
        super().__init__()
        self.save_hyperparameters()
        self.model = UNITE(
            num_cls=num_cls,
            num_heads=num_heads,
            max_len=max_len,
            dropout=dropout,
            encoder_model=encoder_model,
            use_bfloat=use_bfloat,
            cpu_preprocess=cpu_preprocess
        )
        self.ce_loss = nn.CrossEntropyLoss()
        self.ad_loss = ADLoss(
            num_cls=num_cls,
            num_heads=num_heads,
            max_len=max_len,
            delta_within=delta_within,
            delta_between=delta_between,
            eta=eta,
        )
        self.lambda_1 = lambda_1
        self.lambda_2 = lambda_2

        self.lr = lr
        self.decay_steps = decay_steps # Set this in respect to batch size; original batch size was 32

        MetricType = TypedDict('MetricType', {
            "task": Literal['multiclass'] | Literal['binary'],
            "num_classes": int,
            "average": Literal['macro'],
        })

        metric_param: MetricType = {"task": "multiclass", "num_classes": num_cls, "average": "macro"}

        metrics = MetricCollection([
            Accuracy(**metric_param), AveragePrecision(**metric_param), Precision(**metric_param), Recall(**metric_param)
        ])

        self.val_metrics = metrics.clone(prefix="val/")
        self.test_metrics = metrics.clone(prefix="test/")

        self.confmat = ConfusionMatrix(task='multiclass', num_classes=num_cls)
        self.num_cls = num_cls

        self.val_embeds = []
        self.val_labels = []

    def forward(self, x):
        return self.model(x)

    def on_save_checkpoint(self, checkpoint):
        """체크포인트 저장 시 Frozen 파라미터(Backbone)를 제외합니다."""
        state_dict = checkpoint['state_dict']

        # 저장하지 않을 키 필터링 (vis_encoder 관련 키들 제거)
        # 키 이름은 모델 구조에 따라 'model.vis_encoder.'로 시작합니다.
        keys_to_remove = [k for k in state_dict.keys() if "model.vis_encoder" in k]

        for k in keys_to_remove:
            del state_dict[k]

    def on_load_checkpoint(self, checkpoint):
        """로드 시 체크포인트에 없는 백본 가중치를 현재 모델에서 복사해서 채워줌"""
        state_dict = checkpoint['state_dict']
        model_state_dict = self.state_dict()

        # 현재 모델(self.model.vis_encoder)은 이미 __init__에서
        # from_pretrained로 로드된 상태입니다.
        # 체크포인트에 없는 키(백본 가중치)들을 모델의 현재 가중치로 채워줍니다.
        for key in model_state_dict:
            if key not in state_dict:
                state_dict[key] = model_state_dict[key]

    def training_step(self, batch, batch_idx):
        x, y = batch
        logit, P = self.model(x, return_ad_param=True)
        loss_ad, within, between = self.ad_loss(P, y, log_detail=True)
        loss_ce = self.ce_loss(logit, y)
        loss = loss_ce * self.lambda_1 + loss_ad * self.lambda_2


        self.log_dict({
            "train/loss_ad": loss_ad,
            "train/loss_ad/loss_within": within,
            "train/loss_ad/loss_between": between,
            "train/loss_ce": loss_ce,
            "train/loss": loss,
        }, logger=True)


        return loss


    def validation_step(self, batch, batch_idx):
        x, y = batch
        logit, P = self.model(x, return_ad_param=True)
        loss_ad = self.ad_loss(P, y)
        loss_ce = self.ce_loss(logit, y)
        loss = loss_ce * self.lambda_1 + loss_ad * self.lambda_2
        self.log("val/loss_ad", loss_ad, logger=True)
        self.log("val/loss_ce", loss_ce, logger=True)
        self.log("val/loss", loss, prog_bar=True, logger=True)

        self.val_metrics.update(logit, y)
        self.log_dict(self.val_metrics, logger=True)


    def on_test_epoch_start(self):
        self.test_preds = []
        self.test_labels = []

    def test_step(self, batch, batch_idx):
        x, y = batch
        logit = self.model(x)
        preds = torch.argmax(logit, dim=1)

        self.test_preds.append(preds.detach())
        self.test_labels.append(y.detach())

        self.test_metrics.update(logit, y)
        self.log_dict(self.test_metrics, logger=True)

    def on_test_epoch_end(self):
        all_preds = torch.cat(self.test_preds)
        all_labels = torch.cat(self.test_labels)
        conf_matrix = self.confmat(all_preds, all_labels)

        # Plot confusion matrix
        fig = plt.figure(figsize=(8, 6))
        sns.heatmap(
            conf_matrix.cpu().numpy(),
            annot=True,
            fmt='d',
            cmap='Blues',
            xticklabels=[f'Class {i}' for i in range(self.num_cls)],
            yticklabels=[f'Class {i}' for i in range(self.num_cls)]
        )
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix')
        self.logger.experiment.log({"Confusion Matrix": wandb.Image(fig)})
        plt.close(fig)

    def configure_optimizers(self):
        optim = torch.optim.AdamW(self.model.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.StepLR(optim, self.decay_steps, gamma=0.5)

        return {
            "optimizer": optim,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
            },
        }

In [10]:
import os
import math
import cv2
import torch
from pathlib import Path
from typing import Sequence
from collections import Counter
from torch.utils.data import Dataset
from torchvision.io import read_image, ImageReadMode
from torchcodec.decoders import VideoDecoder
import transformers

class CelebDFBaseDataset(Dataset):
    """
    Celeb-DF 데이터셋의 공통 기능을 담은 부모 클래스
    """
    def __init__(
            self,
            paths: Sequence[Path | str],
            length=32, size=(384, 384),
            transform=None,
            device='cpu',
            encoder_model="google/siglip2-base-patch16-224",
            cpu_preprocess=False,
        ):
        self.length = length
        self.size = size
        self.transform = transform
        self.device = device
        self.samples = []

        print(f"Processing {len(paths)} paths...")
        self._prepare_samples(paths)
        print(f"Loaded {len(self.samples)} samples from {len(paths)} files/directories.")

        self.preprocessor = None
        self.cpu_preprocess = cpu_preprocess
        if cpu_preprocess:
            self.preprocessor = transformers.AutoProcessor.from_pretrained(
                encoder_model,
                use_fast=True
            )

    def _get_label(self, path: str) -> int | None:
        """폴더명 기반 레이블 결정"""
        rel_path = path.replace("\\", "/")
        if "YouTube-real" in rel_path or "Celeb-real" in rel_path:
            return 0
        elif "Celeb-synthesis" in rel_path:
            return 1
        return None

    def _prepare_samples(self, paths: Sequence[Path | str]):
        """상속받는 클래스에서 각자의 방식으로 samples 리스트를 채움"""
        for path in paths:
            path_str = str(path)
            label = self._get_label(path_str)
            if label is None:
                continue

            frame_cnt = self._get_frame_count(path_str)
            if frame_cnt <= 0:
                continue

            num_chunks = self._calculate_num_chunks(frame_cnt)
            for i in range(num_chunks):
                self.samples.append({
                    "path": path_str,
                    "chunk_idx": i,
                    "label": label,
                    "total_frames": frame_cnt
                })

    def _get_frame_count(self, path: str) -> int:
        """자식 클래스에서 구현: 전체 프레임 수 반환"""
        raise NotImplementedError

    def _calculate_num_chunks(self, frame_cnt: int) -> int:
        """자식 클래스에서 구현: 프레임 수에 따른 청크 개수 계산"""
        raise NotImplementedError

    def get_label_counter(self) -> Counter:
        return Counter([x['label'] for x in self.samples])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        """자식 클래스에서 구현: 실제 데이터 로드"""
        raise NotImplementedError


In [11]:
class CelebDFVideoDataset(CelebDFBaseDataset):
    """
    비디오 파일(.mp4 등)에서 직접 프레임을 추출하는 데이터셋
    """
    def _get_frame_count(self, path: str) -> int:
        cap = cv2.VideoCapture(path)
        cnt = 0
        if cap.isOpened():
            cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            cap.release()
        return cnt

    def _calculate_num_chunks(self, frame_cnt: int) -> int:
        # Stride=2를 고려한 유효 프레임 기반 계산
        effective_frames = math.ceil(frame_cnt / 2)
        return math.ceil(effective_frames / self.length)

    def __getitem__(self, idx):
        meta = self.samples[idx]
        video_path, chunk_idx, label = meta["path"], meta["chunk_idx"], meta["label"]

        try:
            decoder = VideoDecoder(video_path, device=self.device)
            total_frames = decoder.metadata.num_frames or 100000

            # Stride 2 적용하여 인덱스 계산
            start_frame = chunk_idx * self.length * 2
            indices = [min(start_frame + (i * 2), total_frames - 1) for i in range(self.length)]

            frames_batch = decoder.get_frames_at(indices=indices)
            frames_tensor = frames_batch.data  # (N, C, H, W)

            if self.cpu_preprocess:
                assert self.preprocessor is not None
                processed = self.preprocessor(images=frames_tensor, return_tensors="pt")
                frames_tensor = processed.pixel_values  # Shape: (N, C, H, W)

            # (N, C, H, W) -> (C, N, H, W)
            frames_tensor = frames_tensor.permute(1, 0, 2, 3)

            if self.transform:
                # Video transform이 필요한 경우 여기서 처리 (Batch 단위 지원 필요)
                pass

        except Exception as e:
            print(f"Error loading {video_path}: {e}")
            if self.cpu_preprocess:
                frames_tensor = torch.zeros((3, self.length, self.size[1], self.size[0]), dtype=torch.float32)
            else:
                frames_tensor = torch.zeros((3, self.length, self.size[1], self.size[0]), dtype=torch.uint8)



        return frames_tensor.contiguous(), torch.tensor(label)

In [12]:
class CelebDFImageDataset(CelebDFBaseDataset):
    """
    이미지 파일이 저장된 폴더에서 프레임을 읽어오는 데이터셋
    """
    def _get_frame_count(self, path: str) -> int:
        try:
            return len([f for f in os.listdir(path) if f.endswith(('.jpg', '.png'))])
        except Exception:
            return 0

    def _calculate_num_chunks(self, frame_cnt: int) -> int:
        # 이미 전처리가 완료된 상태이므로 stride 없이 계산
        return math.ceil(frame_cnt / self.length)

    def __getitem__(self, idx):
        meta = self.samples[idx]
        folder_path, chunk_idx, label, total_frames = meta["path"], meta["chunk_idx"], meta["label"], meta["total_frames"]

        frames_list = []
        start_frame_idx = chunk_idx * self.length

        for i in range(self.length):
            current_idx = min(start_frame_idx + i, total_frames - 1)
            file_name = f"frame_{current_idx + 1:06d}.jpg"
            img_path = os.path.join(folder_path, file_name)

            try:
                img_tensor = read_image(img_path, mode=ImageReadMode.RGB)
                if self.transform:
                    img_tensor = self.transform(img_tensor)
                frames_list.append(img_tensor)
            except Exception:
                frames_list.append(torch.zeros((3, self.size[1], self.size[0]), dtype=torch.uint8))

        if frames_list:
            frames_tensor = torch.stack(frames_list, dim=0) # (N, C, H, W)
            if self.cpu_preprocess:
                assert self.preprocessor is not None
                processed = self.preprocessor(images=frames_tensor, return_tensors="pt")
                frames_tensor = processed.pixel_values  # Shape: (N, C, H, W)
            frames_tensor = frames_tensor.permute(1, 0, 2, 3) # (C, N, H, W)
        else:
            if self.cpu_preprocess:
                frames_tensor = torch.zeros((3, self.length, self.size[1], self.size[0]), dtype=torch.float32)
            else:
                frames_tensor = torch.zeros((3, self.length, self.size[1], self.size[0]), dtype=torch.uint8)

        return frames_tensor.contiguous(), torch.tensor(label)

In [13]:
import subprocess
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
from tqdm.notebook import tqdm
from shutil import copy2
import os

def resize_single_video(item):
    src_path, dst_path, size = item
    # 폴더가 없으면 생성
    dst_path.parent.mkdir(parents=True, exist_ok=True)

    # FFmpeg 명령어 구성
    # -y: 기존 파일 덮어쓰기
    # -i: 입력 파일
    # -vf: 비디오 필터 (리사이즈)
    # -c:v libx264: H.264 코덱 사용
    # -crf 23: 일반적인 화질 설정 (낮을수록 고화질)
    # -preset veryfast: 속도 우선 인코딩
    # -an: 오디오 제거 (학습에 필요 없음, 용량 절감)
    cmd = [
        'ffmpeg', '-y',
        '-hwaccel', 'cuda',             # 디코딩 가속 유지
        '-i', str(src_path),
        # 성공했던 코드의 필터 형식을 그대로 사용 (flags=lanczos 등)
        '-vf', f"scale={size[0]}:{size[1]}:flags=lanczos",
        '-c:v', 'libx264',              # NVENC 대신 검증된 libx264 사용
        '-preset', 'fast',              # 속도 조절
        '-crf', '23',                   # 화질 설정
        '-pix_fmt', 'yuv420p',          # 재생 호환성을 위해 추가
        '-an',                          # 오디오 제거
        str(dst_path)
    ]

    # 실행 (로그는 숨김)
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    # ... (cmd 설정 부분)


def preprocess_celebdf(src_root, dst_root, size=(384, 384), max_workers=8):
    src_root = Path(src_root)
    dst_root = Path(dst_root)

    if dst_root.exists():
        print(f"Target directory {dst_root} already exists. Skipping preprocessing.")
        return

    # 모든 mp4 파일 찾기
    video_files = list(src_root.glob("*/*.mp4"))
    tasks = []

    for src_path in video_files:
        rel_path = src_path.relative_to(src_root)
        dst_path = dst_root / rel_path
        if not dst_path.exists():
            tasks.append((src_path, dst_path, size))

    print(f"Starting preprocessing: {len(tasks)} videos...")

    # 멀티프로세싱으로 병렬 처리
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(resize_single_video, tasks, chunksize=4), total=len(tasks)))
    copy2(src_root / "List_of_testing_videos.txt", dst_root / "List_of_testing_videos.txt")

    print("Preprocessing completed.")

In [14]:
import subprocess
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
from tqdm.notebook import tqdm
from shutil import copy2
import os

def extract_frames_single_video(item):
    src_path, dst_dir, size = item

    if dst_dir.exists() and any(dst_dir.iterdir()):
        return

    dst_dir.mkdir(parents=True, exist_ok=True)

    output_pattern = str(dst_dir / "frame_%06d.jpg")

    # FFmpeg 명령어 변경
    # -vf "select='not(mod(n\,2))',scale=...":
    #   1. select: 짝수 프레임(0, 2, 4...)만 선택
    #   2. scale: 리사이즈
    # -vsync vfr: 선택된 프레임만 출력 (타임스탬프 유지를 위해 빈 프레임 생성 방지)
    cmd = [
        'ffmpeg', '-y',
        '-hwaccel', 'cuda',
        '-i', str(src_path),
        '-vf', rf"select='not(mod(n\,2))',scale={size[0]}:{size[1]}:flags=lanczos",
        '-vsync', 'vfr', # Variable Frame Rate: 선택된 프레임만 씀
        '-q:v', '2',
        output_pattern,
        '-threads', '1',
        '-hide_banner', '-loglevel', 'error'
    ]

    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def preprocess_celebdf_frames(src_root, dst_root, size=(384, 384), max_workers=8):
    src_root = Path(src_root)
    dst_root = Path(dst_root)

    video_files = list(src_root.glob("*/*.mp4"))
    tasks = []

    print(f"Scanning files...")
    for src_path in video_files:
        rel_path = src_path.relative_to(src_root)
        dst_dir = dst_root / rel_path.with_suffix('')

        if not dst_dir.exists() or not any(dst_dir.iterdir()):
            tasks.append((src_path, dst_dir, size))

    print(f"Starting frame extraction (Even frames only): {len(tasks)} videos...")

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(extract_frames_single_video, tasks, chunksize=1), total=len(tasks)))

    txt_src = src_root / "List_of_testing_videos.txt"
    if txt_src.exists():
        copy2(txt_src, dst_root / "List_of_testing_videos.txt")

    print("Preprocessing completed.")

In [15]:
from torch.utils.data import WeightedRandomSampler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import kagglehub
import pandas as pd

class CelebDFDataModule(L.LightningDataModule):
    def __init__(
            self, length=32, batch_size=32, num_workers=8, size=(384, 384),
            path="/content/preprocessed", from_img=True, val_split_ratio=0.1,
            device='cpu', prefetch_factor=1, pin_memory=True, persistent_worker=True,
            encoder_model="google/siglip2-base-patch16-384", cpu_preprocess=False):
        super().__init__()
        self.save_hyperparameters()
        self.length = length
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.path = Path(path)
        self.dataset_cls: type[CelebDFBaseDataset] = CelebDFImageDataset
        self.dataset_preprocess_method = preprocess_celebdf_frames
        self.from_img = from_img
        if from_img == False:
            self.dataset_cls = CelebDFVideoDataset
            self.dataset_preprocess_method = preprocess_celebdf
        self.prefetch_factor = prefetch_factor
        self.pin_memory = pin_memory
        self.val_split_ratio = val_split_ratio
        self.size = size
        self.persistent_worker = persistent_worker
        self.device = device
        self.encoder_model = encoder_model
        self.cpu_preprocess = cpu_preprocess

    def prepare_data(self):
        path = kagglehub.dataset_download("reubensuju/celeb-df-v2")
        self.dataset_preprocess_method(path, self.path, size=self.size)

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            # 1. 모든 비디오 폴더 경로 가져오기
            # 전처리된 이미지 폴더 구조: root/Celeb-real/id0_0000 ...
            # glob을 이용해 실제 폴더들을 다 찾습니다.
            all_folders = []
            if self.from_img:
                all_folders = [Path(x) for x in self.path.glob("*/*") if os.path.isdir(x)]
            else:
                all_folders = [Path(x) for x in self.path.glob("*/*.mp4")]

            # 2. Test Set 목록 로드 및 제외
            txt_path = self.path / "List_of_testing_videos.txt"
            test_df = pd.read_csv(txt_path, sep=" ", header=None, names=["label", "path"])
            test_paths_set = set()
            if self.from_img:
                # 확장자를 떼고 비교해야 함 (이미지 폴더명은 확장자가 없으므로)
                test_paths_set = set(
                    test_df["path"].apply(lambda x: str(self.path / Path(x).with_suffix('')).replace("\\", "/")).values
                )
            else:
                test_paths_set = set(test_df["path"].apply(lambda x: str(self.path / x)).replace("\\", "/").values)

            train_val_candidates = []
            for folder in all_folders:
                # 경로 정규화
                folder_str = str(folder).replace("\\", "/")
                # 테스트 셋에 포함되지 않은 것만 Train/Val 후보로 등록
                if folder_str not in test_paths_set:
                     train_val_candidates.append(folder_str)

            # 3. 비디오 단위로 Train / Val 분리 (가장 중요!)
            # 여기서 쪼개야 비디오 하나가 통째로 Train이나 Val 한쪽으로만 갑니다.
            train_videos, val_videos = train_test_split(
                train_val_candidates,
                test_size=self.val_split_ratio,
                random_state=42,
                shuffle=True
            )

            print(f"Total Train/Val Videos: {len(train_val_candidates)}")
            print(f"Split result -> Train Videos: {len(train_videos)}, Val Videos: {len(val_videos)}")

            # 4. 데이터셋 인스턴스 생성 (video_paths 주입)
            self.celebdf_train = self.dataset_cls(
                paths=train_videos,
                length=self.length,
                size=self.size,
                device=self.device,
                encoder_model=self.encoder_model,
                cpu_preprocess=self.cpu_preprocess,
            )

            self.celebdf_val = self.dataset_cls(
                paths=val_videos,
                length=self.length,
                size=self.size,
                device=self.device,
                encoder_model=self.encoder_model,
                cpu_preprocess=self.cpu_preprocess,
            )

        if stage == "test" or stage is None:
            # 테스트 셋은 기존 로직(txt 파일 기반)을 유지하거나,
            # 위에서 test_paths_set을 리스트로 변환해 넘겨줘도 됩니다.
            # 여기서는 편의상 기존 클래스가 is_test=True일 때 txt 파일을 읽는 로직을 그대로 쓴다고 가정하거나
            # 혹은 위와 똑같이 리스트를 만들어서 넘겨줍니다.

            # Test 리스트 생성 로직
            txt_path = self.path / "List_of_testing_videos.txt"
            test_df = pd.read_csv(txt_path, sep=" ", header=None, names=["label", "path"])
            test_videos = []
            if self.from_img:
                test_videos = [
                    str(self.path / Path(x).with_suffix('')).replace("\\", "/")
                    for x in test_df["path"].values
                ]
            else:
                test_videos = [str(self.path / Path(x)) for x in test_df["path"].values]
            # 실제 존재하는 폴더만 필터링
            test_videos = [v for v in test_videos if os.path.exists(v)]

            self.celebdf_test = self.dataset_cls(
                paths=test_videos,
                length=self.length,
                size=self.size,
                device=self.device,
                encoder_model=self.encoder_model,
                cpu_preprocess=self.cpu_preprocess,
            )

    def train_dataloader(self):
        # 2. 현재 훈련 셋(Subset) 내의 클래스별 개수 계산
        label_counts = self.celebdf_train.get_label_counter()

        # 3. 클래스별 가중치 계산 (1 / 개수)
        # 예: Real이 100개, Fake가 1000개면 가중치는 Real: 0.01, Fake: 0.001
        class_weights = {label: 1.0 / count for label, count in label_counts.items()}

        # 4. 각 샘플에 대한 가중치 리스트 생성 (Subset의 순서와 일치해야 함)
        sample_weights = [class_weights[sample['label']]
                          for sample in self.celebdf_train.samples]

        # 5. 샘플러 생성
        # num_samples는 보통 학습 데이터 전체 길이로 설정합니다.
        # replacement=True여야 불균형 데이터에서 적은 쪽을 중복해서 뽑아 균형을 맞춥니다.
        sampler = WeightedRandomSampler(
            weights=sample_weights,
            num_samples=len(sample_weights),
            replacement=True
        )

        return DataLoader(
            self.celebdf_train,
            num_workers=self.num_workers,
            batch_size=self.batch_size,
            pin_memory=self.pin_memory,
            persistent_workers=self.persistent_worker,
            prefetch_factor=self.prefetch_factor,
            sampler=sampler,
        )
    def val_dataloader(self):
        return DataLoader(
            self.celebdf_val,
            num_workers=self.num_workers,
            batch_size=self.batch_size,
            pin_memory=self.pin_memory,
            persistent_workers=self.persistent_worker,
        )
    def test_dataloader(self):
        return DataLoader(
            self.celebdf_test,
            num_workers=self.num_workers,
            batch_size=self.batch_size,
            shuffle=False
        )

In [16]:
import wandb
from google.colab import userdata
wandb_key = userdata.get('wandb_api')
wandb.login(key=wandb_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dhnam0502 (dhnam0502-likelion) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [17]:
BATCH_SIZE = 12
DECAY_STEPS = (1000 * 32) // BATCH_SIZE
LENGTH = 32
SIZE = (224, 224)
import os

datamodule = CelebDFDataModule(
    batch_size=BATCH_SIZE,
    num_workers=8,
    prefetch_factor=2,
    device='cpu',
    # prefetch_factor=None,
    pin_memory=True,
    path="/dev/shm/preprocessed",
    from_img=False,
    length=LENGTH,
    size=SIZE,
    persistent_worker=True,
    # persistent_worker=False,
    encoder_model="google/siglip2-base-patch16-224",
    cpu_preprocess=True,

)
lit_classifier = LitUNITEClassifier(
    decay_steps=DECAY_STEPS,
    delta_within=(0.01, -2),
    delta_between=0.5,
    lambda_1=0.5,
    lambda_2=0.5,
    encoder_model="google/siglip2-base-patch16-224",
    use_bfloat=True,
    max_len=LENGTH,
    cpu_preprocess=True,
)
# lit_classifier = torch.compile(lit_classifier)

Loading weights:   0%|          | 0/408 [00:00<?, ?it/s]

In [18]:
datamodule.prepare_data()

Using Colab cache for faster access to the 'celeb-df-v2' dataset.
Target directory /dev/shm/preprocessed already exists. Skipping preprocessing.


In [19]:
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
from lightning.pytorch.callbacks.lr_monitor import LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks import TQDMProgressBar
from lightning.pytorch.profilers import AdvancedProfiler

profiler = AdvancedProfiler(".", "profile")


wandb_logger = WandbLogger(
    project="UNITE_deepfake_classification",
    name="baseline_test_fixed2",
    log_model=False,
)

# ckpt = ModelCheckpoint(monitor="val/acc", mode="max", save_last=True)
ckpt_drive = ModelCheckpoint(dirpath="/content/drive/MyDrive/bootcamp_proj/final/fixed/", monitor="val/MulticlassAccuracy", mode="max", save_last=True)
lr_monitor = LearningRateMonitor(logging_interval='epoch')

trainer =  L.Trainer(
    max_epochs=10,
    # max_steps=100,
    # profiler=profiler,
    logger=wandb_logger,
    callbacks=[ckpt_drive, lr_monitor],
    precision='bf16-mixed',
    log_every_n_steps=50,
    # precision=16,
    # fast_dev_run=True,
)



INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.


In [20]:
wandb.finish()

In [21]:
torch.set_float32_matmul_precision('high')

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


In [ ]:
trainer.fit(lit_classifier, datamodule=datamodule)
# trainer.fit(lit_classifier, datamodule=datamodule, ckpt_path='/content/UNITE_deepfake_classification/93i7lafx/checkpoints/last.ckpt')

Using Colab cache for faster access to the 'celeb-df-v2' dataset.


wandb: WARNING The anonymous setting has no effect and will be removed in a future version.


Target directory /dev/shm/preprocessed already exists. Skipping preprocessing.


Total Train/Val Videos: 6011
Split result -> Train Videos: 5409, Val Videos: 602
Processing 5409 paths...
Loaded 34682 samples from 5409 files/directories.


In [26]:
# del lit_classifier
# del trainer
# del datamodule
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.checkpoint_callback.last_model_path

'/content/drive/MyDrive/bootcamp_proj/final/v2/last-v1.ckpt'

In [ ]:
trainer.checkpoint_callback.best_model_path

'/content/drive/MyDrive/bootcamp_proj/final/v2/epoch=4-step=7890.ckpt'

In [ ]:
lit_classifier = LitUNITEClassifier.load_from_checkpoint(trainer.checkpoint_callback.last_model_path, strict=False)

In [ ]:
trainer.test(model=lit_classifier, datamodule=datamodule)

Using Colab cache for faster access to the 'celeb-df-v2' dataset.
Scanning files...
Starting frame extraction (Even frames only): 0 videos...


0it [00:00, ?it/s]

Preprocessing completed.
Processing metadata for Test set...


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loaded 3359 samples from 6529 directories.


Output()

/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples found
in target, recall is undefined. Setting recall to one for all thresholds.
  warnings.warn(*args, **kwargs)

/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: Average precision score 
for one or more classes was `nan`. Ignoring these classes in macro-average
  warnings.warn(*args, **kwargs)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │     0.861244797706604     │
│          test/ap          │    0.9100767970085144     │
│      test/precision       │    0.9124864935874939     │
│        test/recall        │     0.861244797706604     │
└───────────────────────────┴───────────────────────────┘

[{'test/acc': 0.861244797706604,
  'test/ap': 0.9100767970085144,
  'test/precision': 0.9124864935874939,
  'test/recall': 0.861244797706604}]

In [ ]:
!ffmpeg -hwaccels | grep cuda

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
!cp /content/UNITE_deepfake_classification/93i7lafx/checkpoints/last.ckpt /content/drive/MyDrive/bootcamp_proj/final/v2/last.ckpt

In [ ]:
from collections import Counter

def count_labels(dataset):
    labels = []
    for _, label in tqdm(dataset):
        labels.append(label.item())
    return Counter(labels)

print("Train set label counts:", count_labels(datamodule.train_dataloader().dataset))
print("Validation set label counts:", count_labels(datamodule.val_dataloader().dataset))
print("Test set label counts:", count_labels(datamodule.test_dataloader().dataset))

  0%|          | 0/34713 [00:00<?, ?it/s]

Train set label counts: Counter({1: 30315, 0: 4398})


  0%|          | 0/3856 [00:00<?, ?it/s]

Validation set label counts: Counter({1: 3374, 0: 482})


  0%|          | 0/3359 [00:00<?, ?it/s]

Test set label counts: Counter({1: 2169, 0: 1190})
